In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.pardir)

In [17]:
import gc

In [10]:
import pdb

In [3]:
import numpy as np
import torch
import umap

In [4]:
from data.dataset import bird_dataset

In [5]:
from utils.utils import load_netG, overlap_encode, overlap_decode, segment_image

# load a bird, get its songs from all days

In [8]:
path2bird = '/home/gagan/data/zudeep/bird_hdfs/r15y2'
dataset = bird_dataset(path2bird)

...total number of folders = 43 ...


In [18]:
X_real = []

for n in range(dataset.nfolders):
    
    seqs = dataset.get(day = n, nsamps=-1)
    for s in seqs:
        segs = segment_image(s, width=16)
        segs = [np.ndarray.flatten(s) for s in segs]
        X_real += segs
        
        

... total available files = 5 ...
... total available files = 220 ...
... total available files = 316 ...
... total available files = 340 ...
... total available files = 644 ...


KeyboardInterrupt: 

In [13]:
len(X_real)

1489096

In [15]:
X_real = np.concatenate(X_real, axis = 1).T

In [16]:
X_real.shape

(23825536, 129)